In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_pickle('test_sessions.df')

In [5]:
train = pd.read_pickle('train_data.df')
train

,seq,len_seq,next
0,"[7261, 7261, 7261, 7261, 7261, 7261, 7261, 726...",1,4682.0
1,"[4682.0, 7261, 7261, 7261, 7261, 7261, 7261, 7...",1,3638.0
2,"[4682.0, 3638.0, 7261, 7261, 7261, 7261, 7261,...",2,107.0
3,"[4682.0, 3638.0, 107.0, 7261, 7261, 7261, 7261...",3,102.0
4,"[4682.0, 3638.0, 107.0, 102.0, 7261, 7261, 726...",4,125.0
...,...,...,...
604817,"[7130.0, 3313.0, 692.0, 3051.0, 4047.0, 5180.0...",7,3376.0
604818,"[7130.0, 3313.0, 692.0, 3051.0, 4047.0, 5180.0...",8,3383.0
604819,"[7130.0, 3313.0, 692.0, 3051.0, 4047.0, 5180.0...",9,3145.0
604820,"[7130.0, 3313.0, 692.0, 3051.0, 4047.0, 5180.0...",10,4390.0


In [3]:
df

,session_id,user_id,item_id,time,watch_ratio,is_pos
248,37,0,2366,1.598921e+09,2.574579,1
249,37,0,3269,1.598929e+09,2.791431,1
250,37,0,4366,1.598929e+09,3.233099,1
251,37,0,4397,1.598930e+09,9.239163,1
252,37,0,1050,1.598930e+09,2.225840,1
...,...,...,...,...,...,...
929538,234773,7172,4515,1.599034e+09,2.008443,1
929539,234773,7172,1094,1.599034e+09,6.080426,1
929683,234844,7174,2398,1.599160e+09,2.897553,1
929684,234844,7174,6968,1.599163e+09,2.387795,1


In [ ]:
eval_sessions = df
eval_ids = eval_sessions.session_id.unique()
groups = eval_sessions.groupby('session_id')
batch, evaluated, total_clicks, total_purchase= 100, 0, 1.0, 0.0
seq_size = 10
item_num = 7261

total_reward = [0, 0, 0, 0]
hit_clicks = [0, 0, 0, 0]
ndcg_clicks = [0, 0, 0, 0]
hit_purchase = [0, 0, 0, 0]
ndcg_purchase = [0, 0, 0, 0]

def pad_history(itemlist,length,pad_item):
    if len(itemlist)>=length:
        return itemlist[-length:]
    if len(itemlist)<length:
        temp = [pad_item] * (length-len(itemlist))
        itemlist.extend(temp)
        return itemlist

while evaluated<len(eval_ids):
    states, len_states, actions, rewards = [], [], [], []
    for i in range(batch):
        id=eval_ids[evaluated]
        group=groups.get_group(id)
        history=[]
        for index, row in group.iterrows():
            state=list(history)
            state = [int(i) for i in state]
            len_states.append(seq_size if len(state)>=seq_size else 1 if len(state)==0 else len(state))
            state=pad_history(state,seq_size,item_num)
            states.append(state)
            action=row['item_id']
            try:
                is_buy=row['t_read']
            except:
                is_buy=row['time']
            reward = 1 if is_buy >0 else 0
            if is_buy>0:
                total_purchase+=1.0
            else:
                total_clicks+=1.0
            actions.append(action)
            rewards.append(reward)
            history.append(row['item_id'])
        evaluated+=1
        if evaluated >= len(eval_ids):
            break